# TinyMS LeNet5 服务器端教程

### 在本教程中，我们会演示使用TinyMS API构建LeNet模型，下载数据集，训练和启动推理服务器的过程。

## 环境要求
 - Ubuntu: `18.04`
 - Python: `3.7.x`
 - Flask: `1.1.2`
 - MindSpore: `CPU-1.1.0`
 - TinyMS: `0.1.0`
 - numpy: `1.17.5`
 - opencv-python: `4.5.1.48`
 - Pillow: `8.1.0`
 - pip: `21.0.1`
 - requests: `2.18.4`
 
## 介绍

TinyMS是一个高级API，目的是让新手用户能够更加轻松地上手深度学习。TinyMS可以有效地减少用户在构建、训练、验证和推理一个模型过程中的操作次数。TinyMS也提供了教程和文档帮助开发者更好的上手和开发。

本教程包括5部分，构建模型、下载数据集、训练，定义servable json和启动服务器，而进行推理操作的部分会在 `LeNet5_Client_tutorial_zh.ipynb`文件中进行讲解。

In [ ]:
import os
import json
import tinyms as ts
import tinyms.optimizers as opt
from tinyms.data import MnistDataset, download_dataset
from tinyms.data.transforms import TypeCast
from tinyms.vision import Inter, Resize, Rescale, HWC2CHW
from tinyms.model import lenet5
from tinyms.model import Model
from tinyms.serving import *
from tinyms.metrics import Accuracy
from tinyms.losses import SoftmaxCrossEntropyWithLogits

## 1. 构建模型

TinyMS封装了MindSpore LeNet5模型中的init和construct函数，代码行数能够大大减少，原有的如下代码段会被极限地压缩到只有两行
```
    import mindspore.nn as nn

    class LeNet5(nn.Cell):
        def __init__(self, num_class=10, num_channel=1):
            super(LeNet5, self).__init__()
            self.conv1 = nn.Conv2d(num_channel, 6, 5, pad_mode='valid')
            self.conv2 = nn.Conv2d(6, 16, 5, pad_mode='valid')
            self.relu = nn.ReLU()
            self.max_pool2d = nn.MaxPool2d(kernel_size=2, stride=2)
            self.flatten = nn.Flatten()
            self.fc1 = nn.Dense(16 * 5 * 5, 120, weight_init=Normal(0.02))
            self.fc2 = nn.Dense(120, 84, weight_init=Normal(0.02))
            self.fc3 = nn.Dense(84, num_class, weight_init=Normal(0.02))

        def construct(self, x):
            x = self.max_pool2d(self.relu(self.conv1(x)))
            x = self.max_pool2d(self.relu(self.conv2(x)))
            x = self.flatten(x)
            x = self.relu(self.fc1(x))
            x = self.relu(self.fc2(x))
            x = self.fc3(x)
            return x

    net = LeNet(class_num=10)
```

In [ ]:
net = lenet5(class_num=10)
model = Model(net)

## 2. 下载数据集

如果根目录下没有创建`mnist`文件夹则MNIST数据集会被自动下载并存放到根目录，如果`mnist`文件夹已经存在于根目录 ，则此步操作会被跳过

In [ ]:
def create_dataset(data_path, batch_size=32, repeat_size=1,
                   num_parallel_workers=1):
    """ create Mnist dataset for train or eval.
    Args:
        data_path: Data path
        batch_size: The number of data records in each group
        repeat_size: The number of replicated data records
        num_parallel_workers: The number of parallel workers
    """
    # 定义数据集
    mnist_ds = MnistDataset(data_path, num_parallel_workers=num_parallel_workers,
                            shuffle=True)

    # 定义map操作
    c_trans = [
        Resize((32, 32), interpolation=Inter.LINEAR),  # 尺寸转换到 (32, 32)
        Rescale(1 / 0.3081, -1 * 0.1307 / 0.3081),  # normalize images
        Rescale(1.0 / 255.0, 0.0),  # rescale images
        HWC2CHW(),  # 从HWC格式转换成CHW格式
    ]
    type_cast_op = TypeCast(ts.int32)  # change data type of label to int32 to fit network

    # apply map operations on images
    mnist_ds = mnist_ds.map(operations=type_cast_op, input_columns="label", num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(operations=c_trans, input_columns="image", num_parallel_workers=num_parallel_workers)
    # apply batch operations
    mnist_ds = mnist_ds.batch(batch_size, drop_remainder=True)
    # apply repeat operations
    mnist_ds = mnist_ds.repeat(repeat_size)

    return mnist_ds

# 下载数据集
if not os.path.exists('/root/mnist'):
    ts.data.download_dataset('mnist', '/root')
else:
    print('************Dataset already exists.**************')

## 3. 训练模型

数据集中的训练集、验证集都会在此步骤中定义，同时也会定义训练参数。训练后生成的ckpt文件会保存到`/etc/tinyms/serving/lenet5`文件夹以便后续使用，训练完成后会进行验证并输出 `Accuracy`指标。

In [ ]:
# 定义训练和验证集
mnist_path = '/root/mnist'
batch_size = 32
train_dataset = create_dataset(os.path.join(mnist_path, "train"), batch_size=batch_size)
eval_dataset = create_dataset(os.path.join(mnist_path, "test"))

# 训练的参数
lr = 0.01
momentum = 0.9
epoch_size = 1
net_loss = SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
net_opt = opt.Momentum(net.trainable_params(), lr, momentum)
net_metrics={"Accuracy": Accuracy()}

model.compile(loss_fn=net_loss, optimizer=net_opt, metrics=net_metrics)
print('************************Start training*************************')
model.train(epoch_size, train_dataset)
model.save_checkpoint('/etc/tinyms/serving/lenet5/lenet5.ckpt')
print('************************Finished training*************************')

model.load_checkpoint('/etc/tinyms/serving/lenet5/lenet5.ckpt')
print('************************Start evaluation*************************')
model.eval(eval_dataset)

## 4. 定义servable.json

定义lenet5 servable json文件，Servable json文件定义了servable名称，模型名称，模型格式和分类数量，以便后续推理使用

In [ ]:
servable_json = [{'name': 'lenet5', 
                  'description': 'This servable hosts a lenet5 model predicting numbers', 
                  'model': {
                      "name": "lenet5", 
                      "format": "ckpt", 
                      "class_num": 10}}]
os.chdir("/etc/tinyms/serving")
json_data = json.dumps(servable_json, indent=4)

with open('servable.json', 'w') as json_file:
    json_file.write(json_data)

## 5. 启动服务器

### 5.1 介绍
TinyMS推理是C/S（Client/Server）架构。TinyMS使用[Flask](https://flask.palletsprojects.com/en/1.1.x/)这个轻量化的网页服务器架构作为C/S通讯的基础架构。为了能够对模型进行推理，用户必须首先启动服务器。如果成功启动，服务器端会监听从地址127.0.0.1，端口号5000发送来的POST请求并且使用MindSpore作为后端来处理这些请求。后端会构建模型，运行推理并且返回结果给客户端

### 5.2 启动服务器

运行下列代码以启动服务器：

In [ ]:
start()

如果用户能够看到类似如下的输出:
```
* Serving Flask app "tinyms.serving.server.server" (lazy loading)
 * Environment: production
   WARNING: This is a development server. Do not use it in a production deployment.
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
```
意味着服务器已经成功启动了，接下来的操作需要到`LeNet5_Client_tutorial_zh.ipynb`文件中完成

## 关闭服务器

关闭服务器，如果使用终端，可以直接CTRL + C关闭，如果使用Jupyter，点击上方`Kernel`再点击`Shutdown`